In [ ]:
# needed packages
from ROOT import TFile
from array import array
import numpy as np
import re
import random
import pickle as pk
import pandas as pd
import math
import sys
from glob import glob

In [ ]:
filepath = "../treeFile.root"
f = TFile(filepath, "update")
tree = f.samples
tree.Print()

In [ ]:
import matplotlib
import matplotlib.cm as cm
import mpl_toolkits.mplot3d.art3d as art3d
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
from matplotlib import font_manager
from matplotlib.patches import Rectangle, PathPatch

cube_size = 10.27
    
x_range = (-985.92, +985.92)
y_range = (-257.56, +317.56)
z_range = (-2888.78, -999.1)
x_range_cube = 0, (x_range[1]-x_range[0])/cube_size
y_range_cube = 0, (y_range[1]-y_range[0])/cube_size
z_range_cube = 0, (z_range[1]-z_range[0])/cube_size
x_size = int((x_range[1]-x_range[0])/10.27)
y_size = int((y_range[1]-y_range[0])/10.27)
z_size = int((z_range[1]-z_range[0])/10.27)

def plot_event(X, add_projs=False):
     
    # fill the detector with reco hits and shrink
    detector = np.zeros((7, 7, 7), dtype=bool)
    detector_hitcharges = X.copy()
    colors = np.empty((7,7,7,4), dtype=object)
        
    norm = matplotlib.colors.Normalize(vmin=X.min(), vmax=X.max())
    cmap = cm.YlOrRd
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    detector[X!=0] = True
    
    for i in range(detector.shape[0]):
        for j in range(detector.shape[1]):
            for k in range(detector.shape[2]):
                colors[i,j,k] = m.to_rgba(detector_hitcharges[i,j,k])
    
    fig = plt.figure(figsize=(24, 24))
        
    # first subplot
    #computed_zorder = False
    ax = fig.add_subplot(1,1,1, projection='3d')
    
    # plot
    color = np.array(['#7A88CCC0'])
    #alpha = 1.0 if only_reco else 0.4 if len(true_x)<50 else 0.75 if len(true_x)<100 else 1.0
    edgecolor = '1.0'
    
    sc = ax.voxels(X, facecolors=colors, edgecolor=edgecolor, alpha=1.0)
    
    cbar = plt.colorbar(m, fraction=0.020, pad=0.2)
    cbar.set_label('# of p.e.', rotation=90, labelpad=19, fontsize=30)
    cbar.ax.tick_params(labelsize=25)
    
    #ax.tick_params(axis='both', which='minor', labelsize=20, length=0)
    ax.set_xlabel('X [cube]', labelpad=50, fontsize=45)
    ax.set_ylabel('Z [cube]', labelpad=50, fontsize=45)
    ax.set_zlabel('Y [cube]', labelpad=50, fontsize=45)
    
    ax.set_xticks([0.5,1.5,2.5,3.5,4.5,5.5,6.5],      minor=True, length=0, width=0, grid_alpha=0)
    ax.set_xticklabels(['1','2','3','4','5','6','7'], minor=True, size=25)
    ax.set_xticklabels([], minor=False)
    
    ax.set_yticklabels([], minor=False)
    ax.set_yticks([0.5,1.5,2.5,3.5,4.5,5.5,6.5],      minor=True, length=0, width=0)
    ax.set_yticklabels(['1','2','3','4','5','6','7'], minor=True, size=25)
    
    ax.set_zticklabels([], minor=False)
    ax.set_zticks([0.5,1.5,2.5,3.5,4.5,5.5,6.5],      minor=True)
    ax.set_zticklabels(['1','2','3','4','5','6','7'], minor=True, size=25)
    
    # Specify different settings for major and minor grids
    #ax.grid(which = 'minor', alpha = 0.3)
    #ax.grid(which = 'major', alpha = 0.7)
    ax.grid(False)
    plt.show()

In [ ]:
thisID, empty_events = 0, 0
from array import array
for event in tree:
    if thisID%10000==0:
        print("{}".format(thisID))
            
    display3D = event.display1D
    
    sparse_image = np.array([[i, data] for i, data in enumerate(display3D) if data!=0])
    initPosition = np.array([event.initPosition.X(), event.initPosition.Y(), event.initPosition.Z()])
    finalPosition = np.array([event.finalPosition.X(), event.finalPosition.Y(), event.finalPosition.Z()])
    initMomentum = np.array([event.initMomentum.Px(), event.initMomentum.Py(), event.initMomentum.Pz(), event.initMomentum.E()])
    
    if sparse_image.shape[0]==0:
        #print("Empty event.")
        empty_events+=1
        
    '''
    plot_event(np.array(display3D).reshape(7,7,7))    
    new_matrix = np.zeros(shape=(7*7*7))
    new_matrix[sparse_image[:,0].astype(int)] = sparse_image[:,1]
    new_matrix = new_matrix.reshape(7,7,7)
    plot_event(new_matrix)
    '''

    np.savez_compressed('images/event{}'.format(thisID),\
                        sparse_image = sparse_image,\
                        initPosition = initPosition,\
                        finalPosition = finalPosition,\
                        initMomentum = initMomentum,\
                       )
        
    #plot_event(display3D)
    thisID+=1

In [ ]:
from array import array

empty_events = 0
for mev in [10,20,30,40,50]:
    thisID = 0
    filepath = "/scratch2/salonso/vertex_activity/fixed_direction/{}MeV/treeFile.root".format(mev)
    f = TFile(filepath, "update")
    tree = f.samples
    tree.Print()

    for event in tree:
        if thisID%10000==0:
            print("Set {}: {}".format(mev, thisID))

        display3D = event.display1D#np.array(event.display1D).reshape(7,7,7)
        
        sparse_image = np.array([[i, data] for i, data in enumerate(display3D) if data!=0])
        initPosition = np.array([event.initPosition.X(), event.initPosition.Y(), event.initPosition.Z()])
        finalPosition = np.array([event.finalPosition.X(), event.finalPosition.Y(), event.finalPosition.Z()])
        initMomentum = np.array([event.initMomentum.Px(), event.initMomentum.Py(), event.initMomentum.Pz(), event.initMomentum.E()])

        if sparse_image.shape[0]==0:
            #print("Empty event.")
            empty_events+=1

        '''
        plot_event(np.array(display3D).reshape(7,7,7))    
        new_matrix = np.zeros(shape=(7*7*7))
        new_matrix[sparse_image[:,0].astype(int)] = sparse_image[:,1]
        new_matrix = new_matrix.reshape(7,7,7)
        plot_event(new_matrix)
        '''

        np.savez_compressed('images_test/{}MeV/event{}'.format(mev, thisID),\
                            sparse_image = sparse_image,\
                            initPosition = initPosition,\
                            finalPosition = finalPosition,\
                            initMomentum = initMomentum,\
                           )

        #display3D = np.array(event.display1D).reshape(7,7,7)
        #plot_event(display3D)
        thisID+=1
        
print(empty_events)

In [ ]:
charges = []
ini_pos = [[], [], []]
ini_P = [[], [], []]
Es = []
Tc = []
thisID = 0

for event in tree:
    if thisID%10000==0:
        print("{}".format(thisID))
            
    display3D = event.display1D
    
    full_image = np.array([[i, data] for i, data in enumerate(display3D) if data!=0])
    initPosition = np.array([event.initPosition.X(), event.initPosition.Y(), event.initPosition.Z()])
    finalPosition = np.array([event.finalPosition.X(), event.finalPosition.Y(), event.finalPosition.Z()])
    initMomentum = np.array([event.initMomentum.Px(), event.initMomentum.Py(), event.initMomentum.Pz(), event.initMomentum.E()])
    
    if full_image.shape[0]==0:
        #print("Empty event.")
        continue
    
    full_image = full_image[:,1]
    e = initMomentum[3]
    initMomentum = initMomentum[:3]
    
    if full_image is not None:
        charges.extend(full_image)
        ini_pos[0].append(initPosition[0])
        ini_pos[1].append(initPosition[1])
        ini_pos[2].append(initPosition[2])
        ini_P[0].append(initMomentum[0])
        ini_P[1].append(initMomentum[1])
        ini_P[2].append(initMomentum[2])
        Es.append(e)
        Tc.append(full_image.sum())
        
    thisID +=1
        
charges = np.array(charges)
ini_pos = np.array(ini_pos)
ini_P = np.array(ini_P)
Es = np.array(Es)
Tc = np.array(Tc)

with open("data.p", "wb") as fd:
    pk.dump([charges, ini_pos, ini_P, Es, Tc], fd)

In [ ]:
Tc.min()